Dans cette partie, on essaie de montrer qu'un réseau de type RBF (peu linéaire) est capable, à défaut de pouvoir être performant contre des exemples adversariaux, de montrer une très faible confiance en ses résultats mauvais.

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.linear_model import LogisticRegression

# Charger les données MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train_flat = x_train.reshape(-1, 28 * 28)
x_test_flat = x_test.reshape(-1, 28 * 28)

# Fonction pour générer des exemples adversariaux (FGSM)
def generate_adversarial_examples(model, x, y, epsilon=0.25):
    adv_examples = []
    for i in range(len(x)):
        x_sample = tf.convert_to_tensor(x[i].reshape(1, 28, 28), dtype=tf.float32)
        y_sample = tf.convert_to_tensor([y[i]], dtype=tf.int64)
        with tf.GradientTape() as tape:
            tape.watch(x_sample)
            pred = model(x_sample)
            loss = tf.keras.losses.sparse_categorical_crossentropy(y_sample, pred)
        gradient = tape.gradient(loss, x_sample)
        perturbation = epsilon * tf.sign(gradient)
        x_adv = x_sample + perturbation
        x_adv = tf.clip_by_value(x_adv, 0, 1)
        adv_examples.append(x_adv.numpy().squeeze())
    return np.array(adv_examples)

# Modèle Maxout simulé avec Dense
def create_maxout_model():
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(256, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Modèle RBF
class ShallowRBFNetwork:
    def __init__(self, num_classes=10, gamma=0.05):
        self.num_classes = num_classes
        self.kmeans = KMeans(n_clusters=num_classes, init='k-means++', max_iter=500, n_init=10)
        self.log_reg = LogisticRegression(max_iter=1000)
        self.gamma = gamma

    def fit(self, x, y):
        self.kmeans.fit(x)
        rbf_features = rbf_kernel(x, self.kmeans.cluster_centers_, gamma=self.gamma)
        self.log_reg.fit(rbf_features, y)

    def predict_proba(self, x):
        rbf_features = rbf_kernel(x, self.kmeans.cluster_centers_, gamma=self.gamma)
        return self.log_reg.predict_proba(rbf_features)

    def predict(self, x):
        return np.argmax(self.predict_proba(x), axis=1)

# Entraînement des modèles
model_maxout = create_maxout_model()
model_maxout.fit(x_train, y_train, epochs=5, verbose=0)

model_rbf = ShallowRBFNetwork()
model_rbf.fit(x_train_flat, y_train)

# Génération des exemples adversariaux avec le modèle Maxout
x_adv = generate_adversarial_examples(model_maxout, x_test, y_test)

# Prédictions sur les exemples adversariaux
y_pred_maxout = np.argmax(model_maxout.predict(x_adv), axis=1)
y_pred_rbf = model_rbf.predict(x_adv.reshape(-1, 28 * 28))

# Calcul des métriques de performance
def calculate_statistics(y_true, y_pred, y_proba):
    error_rate = np.mean(y_true != y_pred) * 100
    confidence_mistakes = np.mean(np.max(y_proba[y_true != y_pred], axis=1)) * 100
    confidence_clean = np.mean(np.max(y_proba[y_true == y_pred], axis=1)) * 100
    return error_rate, confidence_mistakes, confidence_clean

# Résultats pour le réseau RBF
y_proba_rbf = model_rbf.predict_proba(x_adv.reshape(-1, 28 * 28))
error_rate_rbf, confidence_mistakes_rbf, confidence_clean_rbf = calculate_statistics(
    y_test, y_pred_rbf, y_proba_rbf
)

# Affichage des résultats
print("RBF Network Results:")
print(f" - Error rate on adversarial examples: {error_rate_rbf:.2f}%")
print(f" - Average confidence on adversarial mistakes: {confidence_mistakes_rbf:.2f}%")
print(f" - Average confidence on clean examples: {confidence_clean_rbf:.2f}%")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
RBF Network Results:
 - Error rate on adversarial examples: 60.83%
 - Average confidence on adversarial mistakes: 34.78%
 - Average confidence on clean examples: 47.28%
